In [8]:
"""This creates and trains a model given some random data. It can be
used to test your installation of the relevant libraries.
"""

import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from preprocessing import Scaler, load_training_data, transform_to_dx
# from tensorflow.keras import callbacks as cb

tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions
print("Loaded the libraries.")
print(tf.config.list_physical_devices())
MODEL_DIR = "./test_model/"

Loaded the libraries.
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [9]:
# Generate random data
DATA_DIR = "../data/"
DATA_FILE = "training_data.npy"
# DATA_DIR = f"data/GDP/{DT:.0f}day/"


data = load_training_data(DATA_DIR + DATA_FILE, 6000000)  
N = data.shape[0]
print(f"Loaded {N = } datapoints")

data = transform_to_dx(data)


N = data.shape[0]
print(f"{N = }")

X = data[:, 0:2, :].reshape(N, 4)
Y = data[:, 2:4, :].reshape(N, 4)

Y_diff = data[:, 2:4, :] - data[:, 0:2, :] 

print(X.shape)
print(Y.shape)

load (3000000, 4, 2)
load (3000000, 4, 2)
Loaded N = 6000000 datapoints
N = 6000000
(6000000, 4)
(6000000, 4)


In [10]:
# Make simple model
N_C = 32
DT = 4

model = tensorflow.keras.Sequential()
model.add(vgg_model.layers[0])
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    
    # 15 here comes from the number of alphas, the number of mu's and the entries of the covariance matrix.
    
    model = tf.keras.Sequential(
        [tfkl.Dense(256, activation='tanh'),
         tfkl.Dense(256, activation='tanh'),
         tfkl.Dense(256, activation='tanh'),
         tfkl.Dense(256, activation='tanh'),
         tfkl.Dense(512, activation='tanh'),
         tfkl.Dense(512, activation='tanh'),
         tfkl.Dense(N_C * 15, activation=None),
         tfpl.MixtureSameFamily(N_C, tfpl.MultivariateNormalTriL(4))])

print("Built the model.")


# Train model

def nll(data_point, tf_distribution):
    """Negative log likelihood."""
    return -tf_distribution.log_prob(data_point)


LOSS = nll
BATCH_SIZE = 16000
LEARNING_RATE = 1e-3
EPOCHS = 1
OPTIMISER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
VALIDATION_SPLIT = 0.2

model.compile(loss=LOSS, optimizer=OPTIMISER)

History = model.fit(
    X,
    Y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    validation_split=VALIDATION_SPLIT,
    verbose=2,
)
print("Trained the model.")

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_probability.python.layers.distribution_layer.MixtureSameFamily object at 0x7a429b129600> (of type <class 'tensorflow_probability.python.layers.distribution_layer.MixtureSameFamily'>)